In [384]:
import pandas as pd
import numpy as np
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.preprocessing import OneHotEncoder

In [612]:
# Load the data
path = "data/"
mitbih_train = pd.read_csv(path+"mitbih_train.csv")
mitbih_train.columns = list(range(len(mitbih_train.columns)))
mitbih_test = pd.read_csv(path+"mitbih_test.csv")
mitbih_test.columns = list(range(len(mitbih_test.columns)))
mitbih_train[187] = mitbih_train[187].astype(int)
mitbih_test[187] = mitbih_test[187].astype(int)


In [613]:
mitbih_train[187].value_counts()

0    72470
4     6431
2     5788
1     2223
3      641
Name: 187, dtype: int64

In [402]:
# print(len(mitbih_train[mitbih_train[187]==0]))
print(len(mitbih_train[mitbih_train[187]==1]))
mitbih_train0 = mitbih_train[mitbih_train[187]==0]
mitbih_train1 = mitbih_train[mitbih_train[187]==1]

mitbih_train2 = mitbih_train[mitbih_train[187]==2]
mitbih_train4 = mitbih_train[mitbih_train[187]==4]


drop_indices = np.random.choice(mitbih_train0.index, len(mitbih_train0)-2000, replace=False)
mitbih_train = mitbih_train.drop(drop_indices)
drop_indices = np.random.choice(mitbih_train2.index, len(mitbih_train2)-2000, replace=False)
mitbih_train = mitbih_train.drop(drop_indices)
drop_indices = np.random.choice(mitbih_train4.index, len(mitbih_train4)-2000, replace=False)
mitbih_train = mitbih_train.drop(drop_indices)
drop_indices = np.random.choice(mitbih_train1.index, len(mitbih_train1)-2000, replace=False)
mitbih_train = mitbih_train.drop(drop_indices)




print(len(mitbih_train[mitbih_train[187]==0]))
print(len(mitbih_train[mitbih_train[187]==1]))
print(len(mitbih_train))


2000
2000
2000
8641


In [403]:
trainx = mitbih_train.drop([187], axis=1)
trainy = mitbih_train[187]
testx = mitbih_test.drop([187], axis=1)
testy = mitbih_test[187]



In [404]:
"""
Resample the dataset to have equal number of examples for each class. Data resampling using sklearn.
Data augmentation technique changes the value of signal, we are not sure which augmentation technique won't
change the prediction of ECG signal.
"""
print(trainy.value_counts())
sma = SMOTE('not majority', random_state=40)
trainx, trainy = sma.fit_resample(trainx, trainy)
print(trainy.value_counts())
trainx = np.expand_dims(trainx, 2)
testx = np.expand_dims(testx, 2)

# ohe = OneHotEncoder()
# testy = ohe.fit_transform(np.array(testy).reshape(-1,1))
# trainy = ohe.transform(np.array(trainy).reshape(-1,1))



4    2000
2    2000
1    2000
0    2000
3     641
Name: 187, dtype: int64


/Users/dhananjaydave/.pyenv/versions/3.7.4/lib/python3.7/site-packages/imblearn/utils/_validation.py:638: FutureWarning: Pass sampling_strategy=not majority as keyword args. From version 0.9 passing these as positional arguments will result in an error
  FutureWarning)


4    2000
3    2000
2    2000
1    2000
0    2000
Name: 187, dtype: int64


In [405]:
print(trainx.shape)
print(trainy.shape)
print(testx.shape)
print(testy.shape)
trainn, dim, _ = trainx.shape
testn = testy.shape[0]
tags = 5

(10000, 187, 1)
(10000,)
(21891, 187, 1)
(21891,)


In [406]:
#trainx.columns
#trainx_tensors = [torch.tensor(np.array(df)) for df in trainx]
#torch.stack(list_of_tensors)
#print(trainx_tensors[0].size())
#tensor0 = torch.tensor()
#tensor1 = torch.tensor(np.array(trainx.iloc[1]))
# tensor1 = torch.tensor([[np.array(trainx.iloc[0])]])
# tensor2 = torch.tensor([[np.array(trainx.iloc[1])]])
# tensor.size()
# tensory = torch.tensor(trainy)

ttrainx = torch.tensor([np.array(trainx[i]).reshape(1,187) for i in range(trainn)])
ttestx = torch.tensor([np.array(testx[i]).reshape(1,187) for i in range(testn)])
ttesty = torch.tensor(testy)
ttrainy = torch.tensor(trainy)


In [407]:
print(ttrainx.shape)
print(ttrainy.shape)
print(ttestx.shape)
print(ttesty.shape)

torch.Size([10000, 1, 187])
torch.Size([10000])
torch.Size([21891, 1, 187])
torch.Size([21891])


In [282]:
# #print(len(trainy))
# one_hot = torch.zeros(testn, tags)
# one_hot[torch.arange(testn), ttesty] = 1
# ttesty = one_hot

# one_hot = torch.zeros(trainn, tags)
# one_hot[torch.arange(trainn), ttrainy] = 1
# ttrainy = one_hot


# #ttesty = torch.tensor([np.array(testy[i]) for i in range(testn)])

In [283]:
# print(ttesty.size(), ttrainy.size())

torch.Size([21891, 5]) torch.Size([362350, 5])


In [416]:
import torch.utils.data as data_utils

train = data_utils.TensorDataset(ttrainx, ttrainy)
train_loader = data_utils.DataLoader(train, batch_size=500, shuffle=True)


test = data_utils.TensorDataset(ttestx, ttesty)
test_loader = data_utils.DataLoader(test, batch_size=100, shuffle=True)



In [186]:
# # get some random training images
# dataiter = iter(train_loader)
# x, labels = dataiter.next()
# print(x.shape, labels.shape)

torch.Size([50, 187]) torch.Size([50])


In [528]:
# Create the model described in paper
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv1d(1, 32, kernel_size=5)

        self.conv11 = nn.Conv1d(32, 32, kernel_size=5, padding=2)
        self.conv12 = nn.Conv1d(32, 32, kernel_size=5, padding=2)

        self.conv21 = nn.Conv1d(32, 32, kernel_size=5, padding=2)
        self.conv22 = nn.Conv1d(32, 32, kernel_size=5, padding=2)

        self.conv31 = nn.Conv1d(32, 32, kernel_size=5, padding=2)
        self.conv32 = nn.Conv1d(32, 32, kernel_size=5, padding=2)

        self.conv41 = nn.Conv1d(32, 32, kernel_size=5, padding=2)
        self.conv42 = nn.Conv1d(32, 32, kernel_size=5, padding=2)

        self.conv51 = nn.Conv1d(32, 32, kernel_size=5, padding=2)
        self.conv52 = nn.Conv1d(32, 32, kernel_size=5, padding=2)

        self.flatten1 = nn.Flatten()

        self.fc1 = nn.Linear(64, 32)
        self.fc2 = nn.Linear(32, 32)
        
    def do_next(self, x, conv1, conv2):
        x1 = x
        x = conv1(x)
        x = F.relu(x)
        x = conv2(x)
        x = torch.add(x1, x)
        x = F.relu(x)
        x = F.max_pool1d(x, kernel_size=5, stride=2)
        return x

    def forward(self, inp):
        x = self.conv1(inp)
        x = self.do_next(x, self.conv11, self.conv12)
        x = self.do_next(x, self.conv21, self.conv22)
        x = self.do_next(x, self.conv31, self.conv32)
        x = self.do_next(x, self.conv41, self.conv42)
        x = self.do_next(x, self.conv51, self.conv52)
        x = self.flatten1(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x


In [529]:
class FiveNet(Net):
    def __init__(self):
        super(FiveNet, self).__init__()
        self.fc3 = nn.Linear(32, 5)

    def forward(self, inp):
        x = super(FiveNet, self).forward(inp)
        #print(x)
        x = self.fc3(x)
        #output = F.softmax(x, dim=1)
        return x


In [515]:
import copy
fivenet = FiveNet()
# adam = optim.Adam(fivenet.parameters(), lr=0.001)
# criterion = nn.CrossEntropyLoss()
best_model_wts = copy.deepcopy(my_nn.state_dict())
fivenet.load_state_dict(best_model_wts)


<All keys matched successfully>

In [516]:
print(model_test(fivenet, test_loader))

Accuracy of the network on the 21891 test inputs: 86.3140103238774 %%
86.3140103238774


In [550]:
my_nn = FiveNet()
#print(trainx[0].shape)
#print(tensor.size())
p = my_nn(tensor.float())
#p = my_nn(torch.tensor(np.array(trainx[0])).float())

print(p)

tensor([[-1.5339e-01,  1.2765e-01, -1.0228e-01, -1.4452e-02,  9.3743e-02],
        [-1.3959e-01,  1.2678e-01, -1.1014e-01,  4.2389e-04,  7.2181e-02],
        [-1.3961e-01,  1.3812e-01, -9.8849e-02,  9.6237e-05,  1.0807e-01],
        [-1.4908e-01,  1.3826e-01, -8.9222e-02, -1.3549e-02,  1.0736e-01],
        [-1.5443e-01,  1.3843e-01, -8.9461e-02, -2.1898e-03,  9.8225e-02],
        [-1.3709e-01,  1.5392e-01, -8.2855e-02, -1.5285e-03,  1.0699e-01]],
       grad_fn=<AddmmBackward>)


In [527]:
p = twonn(tensor.float())
#p = my_nn(torch.tensor(np.array(trainx[0])).float())

print(p)
p = my_nn(tensor.float())
#p = my_nn(torch.tensor(np.array(trainx[0])).float())

print(p)

# p = my_nn(tensor2.float())
# print(p)

tensor([[-0.1132, -0.2509, -2.4716,  4.5064,  2.1217,  1.0548, -1.3551, -1.3960,
         -2.4039,  0.5018, -1.0359, -1.0365, -4.3443,  0.6044,  3.2484,  0.7197,
          4.7650,  7.8272,  2.4695,  5.8419, -2.1150, -1.4809,  2.4006,  0.5666,
         -1.8740, -3.7884,  0.1279, -2.1668, -2.4403, -5.8466,  2.8172,  4.3611],
        [ 1.5620, -3.8722, -0.7775,  0.3315,  3.9262,  2.4394, -5.0029, -3.4208,
         -3.5437,  1.6339, -2.8805, -0.4099, -5.9477,  2.9163,  3.8448,  1.1345,
          4.6027,  7.1737,  0.6896,  3.7839,  1.2704, -0.3600,  2.4920, -1.6533,
          0.4535, -3.8329, -0.4199, -0.0648, -4.6679, -7.8758,  3.5919,  2.5132],
        [ 2.0435, -3.3133, -1.7740,  1.4034,  2.8841,  0.4741, -4.7121, -4.3640,
         -1.2834,  1.1913, -0.9667, -0.6064, -6.4064,  2.1131,  4.4597,  1.9161,
          4.4311,  4.9684,  0.9270,  3.0831,  1.1098, -0.3981,  2.5687, -2.2672,
         -1.1763, -4.7349,  0.3055, -0.2109, -3.8875, -6.1572,  2.1901,  0.3564],
        [ 0.0458,  0.2767

In [530]:
def model_test(net, testloader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            inputs, labels = data
            outputs = net(inputs.float())
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy test inputs: {} %%'.format(100 * correct / total))
    return 100 * correct / total

In [551]:
my_nn = FiveNet()

adam = optim.Adam(my_nn.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
epoch = 100
loss = 0.4
#trainn/500
training_loss = []
test_acc = []
running_loss = 0.0

In [552]:

for e in range(100):
    print('[%d] loss: %.3f' %
          (e + 1, running_loss ))
    tacc = model_test(my_nn, test_loader)
    training_loss.append(running_loss )
    test_acc.append(tacc)
    running_loss = 0.0
    #print(test_acc)
    if test_acc[-1] > 90:
        print("early stopping! Accuracy on test dataset > 90%")
        break
    running_loss = 0.0

    for i, data in enumerate(train_loader, 0):
        #print(type(i), type(data))
        inputs, labels = data
        #print(inputs.size(), labels.size())
        adam.zero_grad()
        outputs = my_nn(inputs.float())
        #print(outputs.size())
        loss = criterion(outputs, labels)
        loss.backward()
        adam.step()
        running_loss += loss.item()


[1] loss: 0.000
Accuracy test inputs: 7.345484445662601 %%
[2] loss: 31.024
Accuracy test inputs: 24.4895162395505 %%
[3] loss: 24.726
Accuracy test inputs: 62.82490521218766 %%
[4] loss: 19.943
Accuracy test inputs: 51.07121648165913 %%
[5] loss: 16.153
Accuracy test inputs: 62.26303046914257 %%
[6] loss: 13.342
Accuracy test inputs: 64.7663423324654 %%
[7] loss: 11.877
Accuracy test inputs: 78.1919510301037 %%
[8] loss: 10.716
Accuracy test inputs: 73.69238499840117 %%
[9] loss: 9.990
Accuracy test inputs: 84.55529669727285 %%
[10] loss: 9.727
Accuracy test inputs: 63.10355854003929 %%
[11] loss: 8.689
Accuracy test inputs: 77.78995934402266 %%
[12] loss: 8.411
Accuracy test inputs: 87.02206386186104 %%
[13] loss: 8.083
Accuracy test inputs: 77.14129094148281 %%
[14] loss: 8.093
Accuracy test inputs: 72.40875245534694 %%
[15] loss: 7.326
Accuracy test inputs: 82.74633410990818 %%
[16] loss: 7.135
Accuracy test inputs: 69.2933168882189 %%
[17] loss: 6.933
Accuracy test inputs: 89.6989

In [554]:
torch.save(my_nn.state_dict(), "model/model_87")

In [585]:
my_nn = FiveNet()
path = "model/model_91"
my_nn.load_state_dict(torch.load(path))

<All keys matched successfully>

In [456]:
print(testy.value_counts())

0    18117
4     1608
2     1448
1      556
3      162
Name: 187, dtype: int64


In [616]:
print("""
0    4045
1    10505
Name: 187, dtype: int64
""")


0    4045
1    10505
Name: 187, dtype: int64



In [579]:
#tacc = model_test(my_nn, test_loader)
#print(tacc)
from sklearn.metrics import classification_report
allpredicted = []
acc2 = 0
correct = 0
total = 0

with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        outputs = my_nn(inputs.float())
        _, predicted = torch.max(outputs.data, 1)
        allpredicted += predicted
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy test inputs: {} %%'.format(100 * correct / total))

Accuracy test inputs: 91.00543602393678 %%


In [537]:
print(classification_report(allpredicted, testy))

              precision    recall  f1-score   support

           0       0.80      0.83      0.81     17409
           1       0.06      0.03      0.04      1066
           2       0.07      0.07      0.07      1520
           3       0.02      0.01      0.01       307
           4       0.07      0.08      0.08      1589

    accuracy                           0.67     21891
   macro avg       0.20      0.20      0.20     21891
weighted avg       0.65      0.67      0.66     21891



In [443]:
testy2 =testy[testy==2]
a2 = np.array(allpredicted)[testy==2]
print(sum(testy2==a2)/1448)


0.07251381215469613


In [555]:
# Load the data

path = "data/"
ptbdb_abnormal_train = pd.read_csv(path+"ptbdb_abnormal.csv")
ptbdb_normal_train = pd.read_csv(path+"ptbdb_normal.csv")
ptbdb_abnormal_train.columns = list(range(len(ptbdb_abnormal_train.columns)))
ptbdb_normal_train.columns = list(range(len(ptbdb_normal_train.columns)))

ptbdb_abnormal_train[187] = ptbdb_abnormal_train[187].astype(int)
ptbdb_normal_train[187] = ptbdb_normal_train[187].astype(int)


In [556]:
from sklearn.model_selection import train_test_split
print(ptbdb_normal_train.shape)
print(ptbdb_abnormal_train.shape)
normalx = ptbdb_normal_train.drop([187], axis=1)
normaly = ptbdb_normal_train[187]
abnormalx = ptbdb_abnormal_train.drop([187], axis=1)
abnormaly = ptbdb_abnormal_train[187]

ptbdb_allx = np.concatenate((normalx, abnormalx), axis=0)
ptbdb_ally = np.concatenate((normaly, abnormaly), axis=0)

ptrainx, ptestx, ptrainy, ptesty = train_test_split(ptbdb_allx, ptbdb_ally, train_size=0.8)


(4045, 188)
(10505, 188)


In [614]:
from sklearn.model_selection import train_test_split
print(ptbdb_normal_train.shape)
print(ptbdb_abnormal_train.shape)
normalx = ptbdb_normal_train.drop([187], axis=1)
normaly = ptbdb_normal_train[187]
abnormalx = ptbdb_abnormal_train.drop([187], axis=1)
abnormaly = ptbdb_abnormal_train[187]

ptbdb_allx = np.concatenate((normalx, abnormalx), axis=0)
ptbdb_ally = np.concatenate((normaly, abnormaly), axis=0)



(4045, 188)
(10505, 188)


In [557]:
print(np.unique(ptrainy, return_counts=True))
sma = SMOTE('not majority', random_state=40)
ptrainx, ptrainy = sma.fit_resample(ptrainx, ptrainy)
ptrainx = np.expand_dims(ptrainx, 2)
ptestx = np.expand_dims(ptestx, 2)

(array([0, 1]), array([3216, 8424]))


In [558]:
print(np.unique(ptrainy, return_counts=True))
print(ptrainx.shape)
print(ptrainy.shape)
print(ptestx.shape)
print(ptesty.shape)
ptrainn, dim, _ = ptrainx.shape
ptestn = ptesty.shape[0]
tags = 5

(array([0, 1]), array([8424, 8424]))
(16848, 187, 1)
(16848,)
(2910, 187, 1)
(2910,)


In [559]:
pttrainx = torch.tensor([np.array(ptrainx[i]).reshape(1,187) for i in range(ptrainn)])
pttestx = torch.tensor([np.array(ptestx[i]).reshape(1,187) for i in range(ptestn)])
pttesty = torch.tensor(ptesty)
pttrainy = torch.tensor(ptrainy)

In [560]:
ptrain = data_utils.TensorDataset(pttrainx, pttrainy)
ptrain_loader = data_utils.DataLoader(ptrain, batch_size=500, shuffle=True)

ptest = data_utils.TensorDataset(pttestx, pttesty)
ptest_loader = data_utils.DataLoader(ptest, batch_size=100, shuffle=True)

In [601]:
class TwoNet(nn.Module):
    def __init__(self, net):
        super(TwoNet, self).__init__()
        self.net = net
        self.net.fc3 = nn.Linear(32, 2)

    def forward(self, inp):
        #x = super(FiveNet, self.net).forward(inp)
        x = self.net.forward(inp)
        #print(x)
        #x = self.fc3(x)
        return x


In [602]:
print([i.requires_grad for i in my_nn.parameters()])
twonn = TwoNet(my_nn)
print([i.requires_grad for i in twonn.parameters()])
adam = optim.Adam(filter(lambda p: p.requires_grad, twonn.parameters()), lr=0.001)
criterion = nn.CrossEntropyLoss()
epoch = 100
loss = 0.4
#trainn/500
training_loss = []
test_acc = []
running_loss = 0.0

[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, True]
[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, True]


In [611]:
print([i.requires_grad for i in twonn.parameters()])



for e in range(50):
    print('[%d] loss: %.3f' %
          (e + 1, running_loss ))
    tacc = model_test(twonn, ptest_loader)
    training_loss.append(running_loss )
    test_acc.append(tacc)
    running_loss = 0.0
    #print(test_acc)
    if test_acc[-1] > 85:
        print("early stopping! Accuracy on test dataset > 90%")
        break
    running_loss = 0.0

    for i, data in enumerate(ptrain_loader, 0):
        #print(type(i), type(data))
        inputs, labels = data
        #print(inputs.size(), labels.size())
        #adam.zero_grad()
        outputs = my_nn(inputs.float())
        #print(outputs.size())
        loss = criterion(outputs, labels)
        loss.backward()
        adam.step()
        running_loss += loss.item()


[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, True]
[1] loss: 0.000
Accuracy test inputs: 80.10309278350516 %%
[2] loss: 192.580
Accuracy test inputs: 78.21305841924399 %%
[3] loss: 299.057
Accuracy test inputs: 75.53264604810997 %%
[4] loss: 398.564
Accuracy test inputs: 74.5704467353952 %%
[5] loss: 443.175
Accuracy test inputs: 74.5704467353952 %%
[6] loss: 415.536
Accuracy test inputs: 75.25773195876289 %%
[7] loss: 323.567
Accuracy test inputs: 78.00687285223368 %%
[8] loss: 210.440
Accuracy test inputs: 80.44673539518901 %%
[9] loss: 137.443
Accuracy test inputs: 78.14432989690722 %%
[10] loss: 137.401
Accuracy test inputs: 72.98969072164948 %%
[11] loss: 192.464
Accuracy test inputs: 67.42268041237114 %%
[12] loss: 269.104
Accuracy test inputs: 62.61168384879725 %%
[13] loss: 346.585
Accuracy test inputs: 58.76288659793814 %%
[14] loss: 4

In [607]:
torch.save(twonn.state_dict(), "model/two_80")

In [610]:
#tacc = model_test(my_nn, test_loader)
#print(tacc)
from sklearn.metrics import classification_report
allpredicted = []
acc2 = 0
correct = 0
total = 0

with torch.no_grad():
    for data in ptest_loader:
        inputs, labels = data
        outputs = twonn(inputs.float())
        _, predicted = torch.max(outputs.data, 1)
        allpredicted += predicted
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy test inputs: {} %%'.format(100 * correct / total))

print(classification_report(allpredicted, ptesty))


Accuracy test inputs: 80.10309278350516 %%
              precision    recall  f1-score   support

           0       0.26      0.32      0.29       684
           1       0.78      0.73      0.75      2226

    accuracy                           0.63      2910
   macro avg       0.52      0.52      0.52      2910
weighted avg       0.65      0.63      0.64      2910

